# Investor - Flow of Funds - US

### Introduction:

Special thanks to: https://github.com/rgrp for sharing the dataset.

### Step 1. Import the necessary libraries

In [ ]:
using DotEnv
using Pkg
DotEnv.load!()
Pkg.activate(ENV["ENV_PATH"])

using CSV
using Dates
using Plots
using Downloads
using Statistics
using DataFrames

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv). 

### Step 3. Assign it to a variable called df

In [2]:
url = "https://raw.githubusercontent.com/datasets/investor-flow-of-funds-us/master/data/weekly.csv"
file = Downloads.download(url)
df = CSV.read(file, DataFrame)

first(df, 5)

Row,Date,Total Equity,Domestic Equity,World Equity,Hybrid,Total Bond,Taxable Bond,Municipal Bond,Total
,Date,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2012-12-05,-7426,-6060,-1367,-74,5317,4210,1107,-2183
2,2012-12-12,-8783,-7520,-1263,123,1818,1598,219,-6842
3,2012-12-19,-5496,-5470,-26,-73,103,3472,-3369,-5466
4,2012-12-26,-4451,-4076,-375,550,2610,3333,-722,-1291
5,2013-01-02,-11156,-9622,-1533,-158,2383,2103,280,-8931


### Step 4.  What is the frequency of the dataset?

In [3]:
# weekly data

### Step 5. What is the type of the Date ?

In [4]:
eltype(df[!, "Date"])

Date

### Step 6. Set the type of column Date to a Date type

In [5]:
df[!, "Date"] = Date.(df[!, "Date"])
first(df, 5)

Row,Date,Total Equity,Domestic Equity,World Equity,Hybrid,Total Bond,Taxable Bond,Municipal Bond,Total
,Date,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2012-12-05,-7426,-6060,-1367,-74,5317,4210,1107,-2183
2,2012-12-12,-8783,-7520,-1263,123,1818,1598,219,-6842
3,2012-12-19,-5496,-5470,-26,-73,103,3472,-3369,-5466
4,2012-12-26,-4451,-4076,-375,550,2610,3333,-722,-1291
5,2013-01-02,-11156,-9622,-1533,-158,2383,2103,280,-8931


### Step 7.  Change the frequency to monthly, sum the values and assign it to monthly.

In [6]:
# Combine monthly -> sum the value
monthly = deepcopy(df)

function to_period(data::Vector{Date})
    month_ = string.(month.(data))
    for i in 1:length(month_)
        if length(month_[i]) == 1
            month_[i] = "0" * month_[i]
        end
    end
    year_ = year.(data)
    return string.(year_) .* "-" .* month_
end

date = to_period(monthly[!, "Date"])
monthly[!, "Date"] = date

monthly = combine(
    groupby(
        monthly, :Date
    ), 
    "Total Equity" => sum,
    "Domestic Equity" => sum,
    "World Equity" => sum,
    "Hybrid" => sum,
    "Total Bond" => sum,
    "Taxable Bond" => sum,
    "Municipal Bond" => sum,
    "Total" => sum
)
first(monthly, 5)


Row,Date,Total Equity_sum,Domestic Equity_sum,World Equity_sum,Hybrid_sum,Total Bond_sum,Taxable Bond_sum,Municipal Bond_sum,Total_sum
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2012-12,-26156,-23126,-3031,526,9848,12613,-2765,-15782
2,2013-01,3661,-1627,5288,2730,12149,9414,2735,18540
3,2014-04,10842,1048,9794,4931,8493,7193,1300,24267
4,2014-05,-2203,-8720,6518,3172,13767,10192,3576,14736
5,2014-06,2319,-6546,8865,4588,9715,7551,2163,16621


### Step 8. You will notice that it filled the dataFrame with months that don't have any data with NaN. Let's drop these rows.

In [10]:
monthly = dropmissing(monthly)

Row,Date,Total Equity_sum,Domestic Equity_sum,World Equity_sum,Hybrid_sum,Total Bond_sum,Taxable Bond_sum,Municipal Bond_sum,Total_sum
,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2012-12,-26156,-23126,-3031,526,9848,12613,-2765,-15782
2,2013-01,3661,-1627,5288,2730,12149,9414,2735,18540
3,2014-04,10842,1048,9794,4931,8493,7193,1300,24267
4,2014-05,-2203,-8720,6518,3172,13767,10192,3576,14736
5,2014-06,2319,-6546,8865,4588,9715,7551,2163,16621
6,2014-07,-7051,-11128,4078,2666,7506,7026,481,3122
7,2014-08,1943,-5508,7452,1885,1897,-1013,2910,5723
8,2014-09,-2767,-6596,3829,1599,3984,2479,1504,2816
9,2014-11,-2753,-7239,4485,729,14528,11566,2962,12502


### Step 9. Good, now we have the monthly data. Now change the frequency to year.

In [11]:
df[!, "Year"] = year.(df[!, "Date"])

yearly = combine(
    groupby(
        df, :Year
    ),
    "Total Equity" => sum,
    "Domestic Equity" => sum,
    "World Equity" => sum,
    "Hybrid" => sum,
    "Total Bond" => sum,
    "Taxable Bond" => sum,
    "Municipal Bond" => sum,
    "Total" => sum
)

first(yearly, 5)

Row,Year,Total Equity_sum,Domestic Equity_sum,World Equity_sum,Hybrid_sum,Total Bond_sum,Taxable Bond_sum,Municipal Bond_sum,Total_sum
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2012,-26156,-23126,-3031,526,9848,12613,-2765,-15782
2,2013,3661,-1627,5288,2730,12149,9414,2735,18540
3,2014,330,-44689,45021,19570,59890,44994,14896,79787
4,2015,15049,-10459,25508,7280,26028,17986,8041,48357


### BONUS: Create your own question and answer it.